In [1]:
# -*- coding: utf-8 -*-
"""CNN.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1RDQrJejOfArNgzjcXMDQ0C4bBzMljDtA

# Connect with Google Drive
"""

# from google.colab import drive
# drive.mount('/content/drive')

"""# Importing Libraries"""
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import scipy
from scipy.interpolate import interp1d
from scipy.stats import kurtosis
from scipy.stats import skew
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from scipy.ndimage import zoom
from skimage.measure import block_reduce
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.optimizers import Adam
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.utils import to_categorical
from keras.layers.core import Reshape
from keras.layers import LSTM

"""# Loading Data"""

path = "../../Cog_DataSets/testing/"
# path = "drive/MyDrive/Colab Notebooks/CogAge/Datasets/testing/"
test_acc = np.load(path+"testAccelerometer.npy")
test_grav = np.load(path+"testGravity.npy")
test_gyro = np.load(path+"testGyroscope.npy")
test_jinsAcc = np.load(path+"testJinsAccelerometer.npy")
test_jinsGyro = np.load(path+"testJinsGyroscope.npy")
test_Label =np.load(path+"testLabels.npy") 
test_linAcc = np.load(path+"testLinearAcceleration.npy")
test_MsAcc = np.load(path+"testMSAccelerometer.npy")
test_MsGyro = np.load(path + "testMSGyroscope.npy")
test_MsMag = np.load(path+"testMagnetometer.npy")
# test_acc

path = "../../Cog_DataSets/training/"
# path = "drive/MyDrive/Colab Notebooks/CogAge/Datasets/training/"
train_acc = np.load(path+"trainAccelerometer.npy")
train_grav = np.load(path+"trainGravity.npy")
train_gyro = np.load(path+"trainGyroscope.npy")
train_jinsAcc = np.load(path+"trainJinsAccelerometer.npy")
train_jinsGyro = np.load(path+"trainJinsGyroscope.npy")
train_Label =np.load(path+"trainLabels.npy") 
train_linAcc = np.load(path+"trainLinearAcceleration.npy")
train_MsAcc = np.load(path+"trainMSAccelerometer.npy")
train_MsGyro = np.load(path + "trainMSGyroscope.npy")
train_MsMag = np.load(path+"trainMagnetometer.npy")
# print(train_Label.shape)


def Normalize(X):
  norm = []
  for I in range(len(X)):
    norm.append(normalize(X[I]))
  norm=np.array(norm)
  return norm

train_acc = Normalize(train_acc)
train_gyro = Normalize(train_gyro)
train_grav = Normalize(train_grav)
train_linAcc = Normalize(train_linAcc)
train_MsMag = Normalize(train_MsMag)
train_MsAcc = Normalize(train_MsAcc)
train_MsGyro = Normalize(train_MsGyro)
train_jinsAcc = Normalize(train_jinsAcc)
train_jinsGyro = Normalize(train_jinsGyro)

test_acc = Normalize(test_acc)
test_gyro = Normalize(test_gyro)
test_grav = Normalize(test_grav)
test_linAcc = Normalize(test_linAcc)
test_MsMag = Normalize(test_MsMag)
test_MsAcc = Normalize(test_MsAcc)
test_MsGyro = Normalize(test_MsGyro)
test_jinsAcc = Normalize(test_jinsAcc)
test_jinsGyro = Normalize(test_jinsGyro)



# # all data of shape #,400,3
# # adding all relative data.
# # Mobile training accelerometer + Mobile testing accelerometer data
# train_acc_reshaped = np.append(train_acc_reshaped,test_acc_reshaped, axis=0)
# train_gyro_reshaped = np.append(train_gyro_reshaped,test_gyro_reshaped, axis=0)
# train_grav_reshaped = np.append(train_grav_reshaped,test_grav_reshaped, axis=0)
# train_linAcc_reshaped = np.append(train_linAcc_reshaped,test_linAcc_reshaped, axis=0)
# train_MsAcc_reshaped = np.append(train_MsAcc_reshaped,test_MsAcc_reshaped, axis=0)
# train_MsGyro_reshaped = np.append(train_MsGyro_reshaped,test_MsGyro_reshaped, axis=0)
# train_MsMag_reshaped = np.append(train_MsMag_reshaped,test_MsMag_reshaped, axis=0)
# train_jinsAcc_reshaped = np.append(train_jinsAcc_reshaped,test_jinsAcc_reshaped, axis=0)
# train_jinsGyro_reshaped = np.append(train_jinsGyro_reshaped,test_jinsGyro_reshaped, axis=0)


# print("Shape of all sensors after up/down sample... ", train_acc_reshaped.shape, train_gyro_reshaped.shape, train_grav_reshaped.shape, train_linAcc_reshaped.shape
#                        , train_MsAcc_reshaped.shape, train_MsGyro_reshaped.shape, train_MsMag_reshaped.shape,
#                        train_jinsAcc_reshaped.shape, train_jinsGyro_reshaped.shape)

# # all data of shape 4572,400,3
# # 4572 = 2284(training) + 2288(testing)


# # stack
# all_data = np.stack([train_acc_reshaped, train_gyro_reshaped, train_grav_reshaped, train_linAcc_reshaped
#                        , train_MsAcc_reshaped, train_MsGyro_reshaped, train_MsMag_reshaped,
#                        train_jinsAcc_reshaped, train_jinsGyro_reshaped], axis=-1)

# all_Label = np.append(train_Label, test_Label, axis=-1)


2023-05-01 00:31:29.360012: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 00:31:29.735637: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 00:31:29.738456: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 00:31:31.097387: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def downSample(data):
    new_length = 80
    old_length = data.shape[1]

    x_downsampled = np.zeros((data.shape[0], new_length, 3))

    for i in range(2284):
        for j in range(3):
            f = interp1d(np.arange(old_length), data[i, :, j], kind='linear')
            x_downsampled[i, :, j] = f(np.linspace(0, old_length - 1, new_length))
    
    return x_downsampled
            
train_acc_reshaped = downSample(train_acc)
train_gyro_reshaped = downSample(train_gyro)
train_grav_reshaped = downSample(train_grav)
train_linAcc_reshaped = downSample(train_linAcc)
train_MsAcc_reshaped = downSample(train_MsAcc)
train_MsGyro_reshaped = downSample(train_MsGyro)
train_MsMag_reshaped = downSample(train_MsMag)

test_acc_reshaped = downSample(test_acc)
test_gyro_reshaped = downSample(test_gyro)
test_grav_reshaped = downSample(test_grav)
test_linAcc_reshaped = downSample(test_linAcc)
test_MsAcc_reshaped = downSample(test_MsAcc)
test_MsGyro_reshaped = downSample(test_MsGyro)
test_MsMag_reshaped = downSample(test_MsMag)

In [3]:
# all data of shape #,80,3
# adding all relative data.
# Mobile training accelerometer + Mobile testing accelerometer data
train_acc_reshaped = np.append(train_acc_reshaped,test_acc_reshaped, axis=0)
train_gyro_reshaped = np.append(train_gyro_reshaped,test_gyro_reshaped, axis=0)
train_grav_reshaped = np.append(train_grav_reshaped,test_grav_reshaped, axis=0)
train_linAcc_reshaped = np.append(train_linAcc_reshaped,test_linAcc_reshaped, axis=0)
train_MsAcc_reshaped = np.append(train_MsAcc_reshaped,test_MsAcc_reshaped, axis=0)
train_MsGyro_reshaped = np.append(train_MsGyro_reshaped,test_MsGyro_reshaped, axis=0)
train_MsMag_reshaped = np.append(train_MsMag_reshaped,test_MsMag_reshaped, axis=0)
train_jinsAcc_reshaped = np.append(train_jinsAcc,test_jinsAcc, axis=0)
train_jinsGyro_reshaped = np.append(train_jinsGyro,test_jinsGyro, axis=0)


print("Shape of all sensors after up/down sample... ", train_acc_reshaped.shape, train_gyro_reshaped.shape, train_grav_reshaped.shape, train_linAcc_reshaped.shape
                       , train_MsAcc_reshaped.shape, train_MsGyro_reshaped.shape, train_MsMag_reshaped.shape,
                       train_jinsAcc_reshaped.shape, train_jinsGyro_reshaped.shape)

# all data of shape 4572,400,3
# 4572 = 2284(training) + 2288(testing)


# stack
all_data = np.stack([train_acc_reshaped, train_gyro_reshaped, train_grav_reshaped, train_linAcc_reshaped
                       , train_MsAcc_reshaped, train_MsGyro_reshaped, train_MsMag_reshaped,
                       train_jinsAcc_reshaped, train_jinsGyro_reshaped], axis=-1)

all_Label = np.append(train_Label, test_Label, axis=-1)


Shape of all sensors after up/down sample...  (4572, 80, 3) (4572, 80, 3) (4572, 80, 3) (4572, 80, 3) (4572, 80, 3) (4572, 80, 3) (4572, 80, 3) (4572, 80, 3) (4572, 80, 3)


In [4]:

# # 90% training data + labels
# train_data = all_data[: int(all_data.shape[0]*0.9)]
# # 10% testing data + labels
# test_data = all_data[int(all_data.shape[0]*0.9):]
# train_labels = all_Label[: int(all_Label.shape[0]*0.9)]
# test_labels = all_Label[int(all_Label.shape[0]*0.9):]

# print("\nShape of training and testin data + labels...\n")
# print(train_data.shape, test_data.shape)
# print(train_labels.shape, test_labels.shape)


# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(all_data, all_Label, test_size=0.2, random_state=42)

print("Shape of training + validation Data")
print(np.shape(x_train), np.shape(x_val))

# y_train = to_categorical(y_train, num_classes=55)
# y_val = to_categorical(y_val, num_classes=55)

print("Shape of training + validation Labels")
print(np.shape(y_train), np.shape(y_val))


Shape of training + validation Data
(3657, 80, 3, 9) (915, 80, 3, 9)
Shape of training + validation Labels
(3657,) (915,)


In [12]:
# HYPER PARAMTERS

# Filter parameters, i.e. about the number of inputs processed by each neuron of the convolutional layer
filter1Size = (11,1)
filter2Size = (13,1)
filter3Size = (13,1)

# Downsampling factors of the pooling layers
poolingLayer1Factor = (2,1)
poolingLayer2Factor = (3,1)
poolingLayer3Factor = (2,1)

# Number of feature maps processed by each convolutional layer
nbFeaturesLayer1 = 50
nbFeaturesLayer2 = 40
nbFeaturesLayer3 = 30

# Activation function of the convolutional layer(s)
activationConv = 'relu'

# Output dimension of the LSTM
outputLSTM = 55

# Parameters of the dense layer
activationMLP = 'relu'
inputMLP = 100

# Training parameters
batchSize =32
numberOfEpochs = 10
learningRate = 0.001

input_shape = (80,3,9)
nbClasses = 55
timeWindow = 80
nbSensors = 9


#-------------------------------------------------------------------------------------------------------
# convAutoencoder: define a convolutional autoencoder model with 2 conv+pooling layers
#-------------------------------------------------------------------------------------------------------
# def conv2Autoencoder(
#     inputShape,
#     nkerns,
#     filterSizes,
#     poolSizes,
#     activationConv,
#     timeWindow,
#     nbSensors,
#     activationMLP,
#     decoder=True):

#     #outputSizeLastConv = (timeWindow-filterSizes[0][0]+1)/poolSizes[0][0]
#     #inputMLP = nkerns[0]*outputSizeLastConv*nbSensors

#     # NOTE: if padding = 'same', the size of the feature maps doesn't change after being convoluted
#     outputSizeLastConv = timeWindow/poolSizes[0][0]/poolSizes[1][0]
#     inputMLP = nkerns[0]*outputSizeLastConv*nbSensors

#     model = Sequential()

#     print('Input shape: ------------------------------')
#     print(inputShape)

#     # Batch normalization layer
#     model.add(BatchNormalization(input_shape=inputShape))
#     print('Batch norm shape: ------------------------------')
#     print(model.layers[-1].output_shape)

#     # First convolutional layer
#     # Note: default padding = zero padding?
#     model.add(Conv2D(nkerns[0], kernel_size=filterSizes[0], activation=activationConv, padding='same'))
#     print('Conv1 shape: ------------------------------')
#     print(model.layers[-1].output_shape)

#     # Max-pooling layer
#     model.add(MaxPooling2D(pool_size=poolSizes[0], padding='same'))
#     print('Pool1 shape: ------------------------------')
#     print(model.layers[-1].output_shape)

#     # Second convolutional layer
#     # Note: default padding = zero padding?
#     model.add(Conv2D(nkerns[1], kernel_size=filterSizes[1], activation=activationConv, padding='same'))
#     print('Conv2 shape: ------------------------------')
#     print(model.layers[-1].output_shape)

#     # Max-pooling layer
#     model.add(MaxPooling2D(pool_size=poolSizes[1], padding='same'))
#     print('Pool2 shape: ------------------------------')
#     print(model.layers[-1].output_shape)

#     # Dense layer
#     model.add(Flatten())
#     model.add(Dense(inputMLP, activation=activationMLP))
#     print('Dense layer size: ---------------------------')
#     print(model.layers[-1].output_shape)


#     if decoder:

#         # Reshaping layer
#         print('Reshape param: -----------------------------')
#         print(outputSizeLastConv*nbSensors*nkerns[1])
#         model.add(Reshape((int(outputSizeLastConv),int(nbSensors),int(nkerns[1]))))
#         print('Reshape shape: ------------------------------')
#         print(model.layers[-1].output_shape)

#         # First up-sampling layer
#         model.add(UpSampling2D(size=poolSizes[1]))
#         print('Upsampling2 shape: ------------------------------')
#         print(model.layers[-1].output_shape)

#         # First deconvolutional layer
#         model.add(Conv2D(nkerns[1], kernel_size=filterSizes[1], activation=activationConv, padding='same'))
#         print('Deconv2 shape: ------------------------------')
#         print(model.layers[-1].output_shape)

#         # Second up-sampling layer
#         model.add(UpSampling2D(size=poolSizes[0]))
#         print('Upsampling1 shape: ------------------------------')
#         print(model.layers[-1].output_shape)

#         # Second deconvolutional layer
#         model.add(Conv2D(nkerns[0], kernel_size=filterSizes[0], activation=activationConv, padding='same'))
#         print('Deconv1 shape: ------------------------------')
#         print(model.layers[-1].output_shape)

#         # Batch normalization layer
#         model.add(BatchNormalization())
#         print('Batch norm shape: ------------------------------')
#         print(model.layers[-1].output_shape)

#         # Output layer
#         model.add(Conv2D(1, kernel_size=filterSizes[0], activation='linear', padding='same')) 
#         print('Output shape: ------------------------------')
#         print(model.layers[-1].output_shape)      

#     return model

#-------------------------------------------------------------------------------------------------------
# convAutoencoder: define a convolutional autoencoder model with 2 conv+pooling layers
#-------------------------------------------------------------------------------------------------------
def conv2Autoencoder(
    inputShape,
    nkerns,
    filterSizes,
    poolSizes,
    activationConv,
    timeWindow,
    nbSensors,
    activationMLP,
    decoder=True):

    # NOTE: if padding = 'same', the size of the feature maps doesn't change after being convoluted
    outputSizeLastConv = int(np.ceil(timeWindow/poolSizes[0][0]/poolSizes[1][0]))
    inputMLP = nkerns[1]*outputSizeLastConv*nbSensors


    model = Sequential()

    print('Input shape: ------------------------------')
    print(inputShape)

    # Batch normalization layer
    model.add(BatchNormalization(input_shape=inputShape))
    print('Batch norm shape: ------------------------------')
    print(model.layers[-1].output_shape)

    # First convolutional layer
    # Note: default padding = zero padding?
    model.add(Conv2D(nkerns[0], kernel_size=filterSizes[0], activation=activationConv, padding='same'))
    print('Conv1 shape: ------------------------------')
    print(model.layers[-1].output_shape)

    # Max-pooling layer
    model.add(MaxPooling2D(pool_size=poolSizes[0], padding='same'))
    print('Pool1 shape: ------------------------------')
    print(model.layers[-1].output_shape)

    # Second convolutional layer
    # Note: default padding = zero padding?
    model.add(Conv2D(nkerns[1], kernel_size=filterSizes[1], activation=activationConv, padding='same'))
    print('Conv2 shape: ------------------------------')
    print(model.layers[-1].output_shape)

    # Max-pooling layer
    model.add(MaxPooling2D(pool_size=poolSizes[1], padding='same'))
    print('Pool2 shape: ------------------------------')
    print(model.layers[-1].output_shape)

    # Dense layer
    model.add(Flatten())
    model.add(Dense(inputMLP, activation=activationMLP))
    print('Dense layer size: ---------------------------')
    print(model.layers[-1].output_shape)


    if decoder:

        # Reshaping layer
        print('Reshape param: -----------------------------')
        print(outputSizeLastConv*nbSensors*nkerns[1])
        model.add(Reshape((outputSizeLastConv, nbSensors, nkerns[1])))
        print('Reshape shape: ------------------------------')
        print(model.layers[-1].output_shape)

        # First up-sampling layer
        model.add(UpSampling2D(size=poolSizes[1]))
        print('Upsampling2 shape: ------------------------------')
        print(model.layers[-1].output_shape)

        # First deconvolutional layer
        model.add(Conv2D(nkerns[1], kernel_size=filterSizes[1], activation=activationConv, padding='same'))
        print('Deconv2 shape: ------------------------------')
        print(model.layers[-1].output_shape)

        # Second up-sampling layer
        model.add(UpSampling2D(size=poolSizes[0]))
        print('Upsampling1 shape: ------------------------------')
        print(model.layers[-1].output_shape)

        # Second deconvolutional layer
        model.add(Conv2D(nkerns[0], kernel_size=filterSizes[0], activation=activationConv, padding='same'))
        print('Deconv1 shape: ------------------------------')
        print(model.layers[-1].output_shape)

        # Batch normalization layer
        model.add(BatchNormalization())
        print('Batch norm shape: ------------------------------')
        print(model.layers[-1].output_shape)

        # Output layer
        model.add(Conv2D(1, kernel_size=filterSizes[0], activation='linear', padding='same')) 
        print('Output shape: ------------------------------')
        print(model.layers[-1].output_shape)      

    return model



input_shape = (80, 3, 9)
nkerns = [32, 64]
filterSizes = [(3, 3), (3, 3)]
poolSizes = [(2, 2), (2, 2)]
activationConv = 'relu'
timeWindow = input_shape[0]
nbSensors = input_shape[2]
activationMLP = 'relu'

model = conv2Autoencoder(
    inputShape=input_shape,
    nkerns=nkerns,
    filterSizes=filterSizes,
    poolSizes=poolSizes,
    activationConv=activationConv,
    timeWindow=timeWindow,
    nbSensors=nbSensors,
    activationMLP=activationMLP
    )

# model = conv2Autoencoder(inputShape=input_shape,
#                             nkerns=[nbFeaturesLayer1, nbFeaturesLayer2, nbFeaturesLayer3],
#                             filterSizes=[filter1Size, filter2Size, filter3Size],
#                             poolSizes=[poolingLayer1Factor, poolingLayer2Factor, poolingLayer3Factor],
#                             activationConv=activationConv,
#                             timeWindow=timeWindow,
#                             nbSensors=nbSensors,
#                             activationMLP='sigmoid'
#                            )

Input shape: ------------------------------
(80, 3, 9)
Batch norm shape: ------------------------------
(None, 80, 3, 9)
Conv1 shape: ------------------------------
(None, 80, 3, 32)
Pool1 shape: ------------------------------
(None, 40, 2, 32)
Conv2 shape: ------------------------------
(None, 40, 2, 64)
Pool2 shape: ------------------------------
(None, 20, 1, 64)
Dense layer size: ---------------------------
(None, 11520)
Reshape param: -----------------------------
11520
Reshape shape: ------------------------------
(None, 20, 9, 64)
Upsampling2 shape: ------------------------------
(None, 40, 18, 64)
Deconv2 shape: ------------------------------
(None, 40, 18, 64)
Upsampling1 shape: ------------------------------
(None, 80, 36, 64)
Deconv1 shape: ------------------------------
(None, 80, 36, 32)
Batch norm shape: ------------------------------
(None, 80, 36, 32)
Output shape: ------------------------------
(None, 80, 36, 1)


In [13]:
model.compile(
    loss='mse',
    optimizer=Adam(learning_rate=learningRate),
    metrics=['accuracy']
)


# model.fit(x=x_train, y=x_train, epochs=2, batch_size=batchSize, validation_data=(y_train, y_train))
model.fit(all_data, all_Label, epochs=2, batch_size=batchSize, validation_split=0.2)


Epoch 1/2
115/115 [==============================] - 73s 614ms/step - loss: 800.5683 - accuracy: 0.0203 - val_loss: 443.3828 - val_accuracy: 0.0207
Epoch 2/2
115/115 [==============================] - 72s 626ms/step - loss: 393.4245 - accuracy: 0.0205 - val_loss: 734.0956 - val_accuracy: 0.0208


In [14]:
predict_train_comp = model.predict(x_train)
prediction_test_comp = model.predict(x_val)

29/29 [==============================] - 4s 139ms/step


In [ ]:
# model.summary()

# from sklearn.svm import SVC

# # x_train_compressed = model.predict(train_data)
# # x_test_compressed = model.predict(test_data)

# predict_train_comp = np.reshape(predict_train_comp, (predict_train_comp.shape[0], 80*9))
# # Train an SVM classifier on the compressed data
# svm = SVC(kernel='linear', C=1, gamma='auto')
# svm.fit(predict_train_comp, y_train)

# # print(x_train_comp.shape)

In [ ]:
# # Evaluate the accuracy of the SVM classifier
# prediction_test_comp = np.reshape(prediction_test_comp, (prediction_test_comp.shape[0], 80*9))
# svm_score = svm.score(prediction_test_comp, y_val)
# print("Accuracy: %.2f%%" % (svm_score*100))
